PART 1: PHPMyAdmin
You will complete questions 1-9 below in the PHPMyAdmin interface. 
Log in by pasting the following URL into your browser, and
using the following Username and Password:

/* QUESTIONS 

/* Q1: Some of the facilities charge a fee to members, but some do not.
Write a SQL query to produce a list of the names of the facilities that do. */

SELECT name
FROM `Facilities`
WHERE membercost >0;

/* Q2: How many facilities do not charge a fee to members? */

SELECT COUNT( facid )
FROM `Facilities`
WHERE membercost =0;

/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question. */
SELECT facid, name, membercost, monthlymaintenance
FROM `Facilities`
WHERE membercost < 0.2 * monthlymaintenance;

/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. */

SELECT *
FROM `Facilities`
WHERE facid
IN ( 1, 5 );

/* Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question. */

SELECT name, monthlymaintenance,
CASE
WHEN monthlymaintenance >100
THEN "expensive"
ELSE "cheap"
END AS price_cat
FROM Facilities;

/* Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution. */

SELECT surname, firstname
FROM `Members`
WHERE joindate = (
SELECT MAX( joindate )
FROM Members ) AND memid!=0;

/* Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name. */

SELECT DISTINCT(CONCAT(surname, " ", firstname)) AS name, f.name AS court_name FROM `Members` as m
LEFT JOIN Bookings AS b
ON m.memid=b.memid
LEFT JOIN Facilities AS f
ON b.facid=f.facid
WHERE b.facid IN (0,1) AND b.memid!=0
ORDER BY name;

/* Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries. */

SELECT f.name AS facility_name, CONCAT(m.surname, " ", m.firstname) AS member_name, 
CASE WHEN m.memid!=0 THEN membercost*slots
	ELSE guestcost*slots END AS cost 
FROM Bookings AS b
LEFT JOIN Members AS m
ON b.memid=m.memid
LEFT JOIN Facilities AS f
ON  b.facid=f.facid
WHERE DATE(starttime)="2012-09-14"
AND guestcost*slots>30 OR membercost*slots>30
ORDER BY  cost  DESC;

/* Q9: This time, produce the same result as in Q8, but using a subquery. */

SELECT facility_name, member_name, cost
FROM 
(SELECT f.name AS facility_name, CONCAT(m.surname, " ", m.firstname) AS member_name, 
CASE WHEN m.memid!=0 THEN membercost*slots
	ELSE guestcost*slots END AS cost 
 FROM Bookings AS b
 LEFT JOIN Members AS m
ON b.memid=m.memid
 LEFT JOIN Facilities AS f
ON  b.facid=f.facid
 WHERE DATE(starttime)="2012-09-14"
AND guestcost*slots>30 OR membercost*slots>30) AS sub
ORDER BY  cost  DESC

/* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

In [1]:
from sqlalchemy import create_engine
import pandas as pd
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')


In [2]:
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


QUESTIONS:
/* Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members! */

In [3]:
df = pd.read_sql_query("SELECT * FROM (SELECT f.name AS facility_name, CASE WHEN memid!=0 THEN SUM(membercost*slots) ELSE SUM(guestcost*slots) END AS total_revenue  FROM bookings AS b LEFT JOIN Facilities AS f ON b.facid=f.facid GROUP BY f.facid) WHERE total_revenue<1000 ORDER BY total_revenue DESC;", engine)
print(df)   

     facility_name  total_revenue
0  Badminton Court              0
1     Table Tennis              0
2    Snooker Table              0
3       Pool Table              0


#Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order */


In [4]:
df1 = pd.read_sql_query("SELECT m.surname AS member_surname,m.firstname AS member_firstname,   sub.surname AS recommender_surname, sub.firstname AS recommender_firstname FROM Members AS m  LEFT JOIN Members AS sub ON m.recommendedby=sub.memid WHERE m.recommendedby IS NOT NULL AND sub.recommendedby IS NOT NULL ORDER BY m.surname, m.firstname;", engine)
print(df1.head(10))

  member_surname member_firstname recommender_surname recommender_firstname
0          Bader         Florence            Stibbons                Ponder
1          Baker             Anne            Stibbons                Ponder
2          Baker          Timothy             Farrell                Jemima
3         Boothe              Tim              Rownam                   Tim
4        Butters           Gerald               Smith                Darren
5         Coplin             Joan               Baker               Timothy
6        Crumpet            Erica               Smith                 Tracy
7           Dare            Nancy            Joplette                Janice
8        Genting          Matthew             Butters                Gerald
9           Hunt             John             Purview             Millicent


/* Q12: Find the facilities with their usage by member, but not guests */

In [5]:
#NOT SURE WHAT IS MEANT BY THIS TASK. NO OF USAGES PER MEMBER? ONLY TOTAL NO OF USAGES BY ALL MEMBERS; OR HOW OFTEN EACH MEMBERS USED EACH FACILITY?
df2 = pd.read_sql_query("SELECT f.name AS facility_name, COUNT(b.bookid) AS total_no_usages, m.surname, m.firstname FROM Facilities AS f  LEFT JOIN Bookings AS b ON f.facid=b.facid LEFT JOIN Members AS m ON m.memid=b.memid WHERE b.memid!=0 GROUP BY f.facid, b.memid ORDER BY surname;", engine)
print(df2.head(100))

      facility_name  total_no_usages   surname firstname
0    Tennis Court 1                1     Bader  Florence
1    Tennis Court 2                8     Bader  Florence
2   Badminton Court                9     Bader  Florence
3      Table Tennis               42     Bader  Florence
4    Massage Room 2                2     Bader  Florence
..              ...              ...       ...       ...
95       Pool Table                8     Jones     David
96       Pool Table                2     Jones   Douglas
97   Tennis Court 1               19  Joplette    Janice
98   Tennis Court 2                8  Joplette    Janice
99     Table Tennis                9  Joplette    Janice

[100 rows x 4 columns]


/* Q13: Find the facilities usage by month, but not guests */

In [6]:
#SAME DOUBT AS ABOVE
df3 = pd.read_sql_query("SELECT strftime('%m', b.starttime) AS month, f.name AS facility_name, COUNT(b.bookid) AS total_no_usages_month FROM Facilities AS f  LEFT JOIN Bookings AS b ON f.facid=b.facid WHERE b.memid!=0 GROUP BY strftime('%m', b.starttime), f.facid  ORDER BY strftime('%m', b.starttime), COUNT(b.bookid) DESC ;", engine)
print(df3)


   month    facility_name  total_no_usages_month
0     07       Pool Table                    103
1     07   Massage Room 1                     77
2     07    Snooker Table                     68
3     07   Tennis Court 1                     65
4     07  Badminton Court                     51
5     07     Table Tennis                     48
6     07   Tennis Court 2                     41
7     07     Squash Court                     23
8     07   Massage Room 2                      4
9     08       Pool Table                    272
10    08    Snooker Table                    154
11    08   Massage Room 1                    153
12    08     Table Tennis                    143
13    08  Badminton Court                    132
14    08   Tennis Court 1                    111
15    08   Tennis Court 2                    109
16    08     Squash Court                     85
17    08   Massage Room 2                      9
18    09       Pool Table                    408
19    09    Snooker 